In [0]:
import gensim

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Загрузим предобученные векторные представления слов

In [0]:
! wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2020-04-01 06:32:11--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.0.150
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.0.150|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  16.7MB/s    in 97s     

2020-04-01 06:33:48 (16.3 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [0]:
wv_embeddings = gensim.models.KeyedVectors.load_word2vec_format(fname='/content/GoogleNews-vectors-negative300.bin.gz', limit=500000, binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Векторные представления текста: средний вектор всех слов в вопросе.

In [0]:
import numpy as np
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

In [0]:
def question_to_vec(question, embeddings, dim=300):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """
    
    # убрать знак вопроса, если он есть
    words = question.replace("?", "")
    n_known = 0
    result = np.array([0] * dim, dtype=float)
    
    for word in words:
        if word in embeddings:
            result += embeddings[word]
            n_known += 1
            
    if n_known != 0:
        return result/n_known
    else:
        return result

### Ранжирование вопросов StackOverflow

- *тестовая* выборка (test.tsv) содержит в каждой строке: *вопрос, похожий вопрос, отрицательный пример 1, отрицательный пример 2, ...*
TEST!!!

Считаем тестовую выборку 

In [0]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.strip().split('\t'))
    return data

In [0]:
# !wget -O test.tsv https://drive.google.com/open?id=12VAWjs-kvpgapurkuURus7E53F711uFV

In [0]:
test = read_corpus('/content/drive/My Drive/Stepik_Mipt/test.tsv')

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [0]:
import re
import nltk
import string
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Кол-во стоп стопслов

In [0]:
stopWords = set(stopwords.words('english'))
len(stopWords)

179

In [0]:
def text_prepare(text):
    """
        text: a string
        
        return: modified string
    """
    # Переводим символы в нижний регистр
    text = text.lower()
    
    # Заменяем символы пунктуации на пробелы
    text = re.sub(r'[{}]'.format(string.punctuation), ' ', text)
    
    # Удаляем "плохие" символы
    text = re.sub('[^A-Za-z0-9 ]', '', text)
    
    # Удаляем стопслова.
    stopWords = set(stopwords.words('english'))
    for stopWord in stopWords:
        text = re.sub(r'\b{}\b'.format(stopWord), '', text)
    return text

In [0]:
from tqdm import tqdm_notebook
from copy import deepcopy
new_test = deepcopy(test)
for i in range(len(test)):
    for j in range(len(test[i])):
        new_test[i][j] = text_prepare(test[i][j])

Функция, которая на основе косинусного растояния между векторами
 находит к вопросу ближайшие похожие.



In [0]:
# Flatten initial test (list of lists). UNIQUE Strings 
flatten_test_1 = sorted(set([item for sublist in test for item in sublist]), key=[item for sublist in test for item in sublist].index)

In [0]:
def find_closest_questions_test(question, k=5):
    """
    function that finds closest questions from dataset given question
    args:
        question: question, preprocessed using text_prepare 
        k: how many nearest questions to find
    """

    dist_s = []
    vec_question = question_to_vec(question=question, embeddings=wv_embeddings, dim=300)
    
    questions =  sorted(set([item for sublist in new_test for item in sublist]), key=[item for sublist in new_test for item in sublist].index) 

    vec_questions = np.array([question_to_vec(question=questions[i], embeddings=wv_embeddings, dim=300)
                      for i in range(len(questions))])
    dist_s = cosine_similarity(vec_questions, np.array([vec_question]))[:, 0]
 
        
    # Sort dist_s first five values
    sorted_dist_s = sorted(dist_s)[:k] # [::-1][:k]
    # Get indexes of sorted_dist_s
    idx_sorted_dists_s = dist_s.argsort()
    
    # get questions by indexes 
    sorted_questions = np.array(flatten_test_1)[idx_sorted_dists_s.tolist()]
    # print(max(idx_sorted_dists_s))
    sorted_dict = list(zip(sorted_questions[:k], sorted_dist_s))
    return sorted_dict
    


In [0]:
find_closest_questions_test(text_prepare("Why am I so stupid?"))

35411 35411


[('why is 1.2 * 30 = 35?', 0.28602382144890576),
 ("Why is ' 2' > '10'?", 0.33366496621354574),
 ('Why is (18446744073709551615 == -1) true?', 0.38654803470784777),
 ('Why does 6.00 + (plus) 5.00 = "5.006.00"?', 0.48149845925095597),
 ("How to change '1.123.456.789' to '1.123.456,789' in python3?",
  0.5329101084272192)]